In [1]:
import simpy
import random


In [2]:
def incident(env, incident_id, engine,time_engaged,time_of_incident):
     # Incidents arriving
     yield env.timeout(time_of_incident)
     print('%s occurred at %d' % (incident_id, env.now))
     # Request one of the engines
     with engine.get() as req:
         # engine responds to incident
         yield req
         print('%s starting response to %s at %s' % (req.value,incident_id, env.now))
         # incident is cleared
         yield env.timeout(time_engaged)
         print('%s cleared at %s' % (incident_id, env.now))
         engine.put(req.value)

In [3]:
env = simpy.Environment()


## Example with Priority Store

In [4]:
#define engines
engines = simpy.PriorityStore(env,capacity = 2)
engines.put(simpy.PriorityItem(1,'Engine 1'))
engines.put(simpy.PriorityItem(0,'Engine 2'))

<StorePut() object at 0x133766c27c0>

In [5]:
time_engaged = 5
for i in range(4):
    env.process(incident(env, 'Incident %d' % i, engines,time_engaged,i*2))
env.run()

Incident 0 occurred at 0
PriorityItem(priority=0, item='Engine 2') starting response to Incident 0 at 0
Incident 1 occurred at 2
PriorityItem(priority=1, item='Engine 1') starting response to Incident 1 at 2
Incident 2 occurred at 4
Incident 0 cleared at 5
PriorityItem(priority=0, item='Engine 2') starting response to Incident 2 at 5
Incident 3 occurred at 6
Incident 1 cleared at 7
PriorityItem(priority=1, item='Engine 1') starting response to Incident 3 at 7
Incident 2 cleared at 10
Incident 3 cleared at 12


## Example with Priority Resource

In [44]:
engine1 = simpy.PriorityResource(env,capacity = 1)


In [45]:
engine1.request(priority=1)
engine1.request(priority=3)
engine1.request(priority=2)


<PriorityRequest() object at 0x13376784160>

In [46]:
engine1.GetQueue()

[]

In [47]:
engine1.queue

[<PriorityRequest() object at 0x13376784160>,
 <PriorityRequest() object at 0x133767840d0>]

#### requests are queued by priority, not by order of arrival

In [53]:
engine1.queue[0].priority,engine1.queue[1].priority

(2, 3)

## Priority Store with Filter

In [109]:
engines = simpy.FilterStore(env,capacity = 2)


#### This will get engine 2 first because of the filter within get

In [110]:
engines.put('Engine 1')
engines.put('Engine 2')
print(engines.items)
a = engines.get(lambda x:x.split(' ')[1]=='2')


['Engine 1', 'Engine 2']


In [111]:
a.value

'Engine 2'

return Engine 2

In [112]:
engines.put(a.value)
engines.items

['Engine 1', 'Engine 2']

#### this will select engine 1 because no filter is specified

In [113]:
a = engines.get()
a.value

'Engine 1'

#### Insert 3 Engines and 1 Ambulance

In [192]:
class Apparatus:
    def __init__(self,name,apparatus_type,als_bls,crew,base_station):
        self.name       = name
        self.unit_type  = apparatus_type
        self.ems_type   = als_bls
        self.crew_size  = crew
        self.station    = base_station

class Station:
    def __init__(self,name,first_due_district):
        self.name = name
        self.area = first_due_district
        
# class Department

In [184]:
stations = [Station('Station %s' % x, x) for x in range(1,4)]
department_units = simpy.FilterStore(env, capacity = 4)

In [185]:
stations[2].name

'Station 3'

In [186]:
for i, station in enumerate(stations):
    department_units.put(Apparatus('Engine %s' % (i+1),'engine','als', 4, station.name))

In [187]:
print(department_units.items[0].name,department_units.items[0].station)

Engine 1 Station 1


In [188]:
department_units.put(Apparatus('Ambulance 1','ambulance','als',2,stations[0].name))

<StorePut() object at 0x1337761d9d0>

In [189]:
print(department_units.items[-1].name,department_units.items[-1].station)

Ambulance 1 Station 1


In [190]:
incident = department_units.get(lambda x: x.unit_type == 'ambulance' or x.station == 'Station 2')

'Engine 2'